In [1]:
### 週販レポート2025年版

# 標準ライブラリ
from datetime import datetime, timedelta

# サードパーティライブラリ
import pandas as pd
from pandas import Timedelta
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from IPython.display import Markdown, display

# プロジェクト内モジュール
from gamedata import hard_sales as hs
from gamedata import plot_hard as ph
from gamedata import hard_info as hi
from gamedata import hard_event as he


In [12]:
df = hs.load_hard_sales()
df = hs.pivot_sales(df, begin=datetime(2025,1,1))

event_df = he.load_hard_event()
filtered_events = he.add_event_positions(event_df, df, priority=3)
#for report_date, event_row in filtered_events.iterrows():
#    print(event_row)


display(filtered_events)

,event_date,hw,hw2,event_name,priority,x_pos,y_pos
report_date,,,,,,,
2025-01-19,2025-01-16,NSW,,Switch2初公開,2,2025-01-16,47560.0
2025-03-02,2025-02-28,PS5,,MHWs,1,2025-02-28,108978.0
2025-03-02,2025-02-28,PS5,,PS5新生活セール,2,2025-02-28,108978.0
2025-03-30,2025-03-27,NSW,,Nintendo Direct,3,2025-03-27,30489.0
2025-04-06,2025-04-02,NSW,,Switch2 Direct,1,2025-04-02,35041.0
2025-06-01,2025-05-28,PS5,,Days of Play2025,2,2025-05-28,15266.0
2025-06-08,2025-06-05,NSW,,Switch2発売,1,2025-06-05,16611.0
2025-07-20,2025-07-17,NS2,,DKバナンザ,2,2025-07-17,152165.0


In [16]:
def adjust_event(df: pd.DataFrame, threshold: float) -> pd.DataFrame:
    """
    (x_pos, y_pos) が完全一致している行について、
    priority が低い方の y_pos を threshold 増やす。
    priority が同じ場合は変更しない。
    """
    out = df.copy()

    def adjust_group(g: pd.DataFrame) -> pd.DataFrame:
        if len(g) <= 1:
            return g
        min_p = g["priority"].min()
        mask = g["priority"] > min_p
        g.loc[mask, "y_pos"] = g.loc[mask, "y_pos"] + threshold
        return g

    out = (
        out.groupby(["x_pos", "y_pos"], sort=False, group_keys=False)
           .apply(adjust_group)
    )
    return out

df2 = adjust_event(filtered_events, threshold=1000)
df2

/var/folders/p4/348jfj6522l78r8xkkk0csb40000gn/T/ipykernel_6523/2190167338.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(adjust_group)


,event_date,hw,hw2,event_name,priority,x_pos,y_pos
report_date,,,,,,,
2025-01-19,2025-01-16,NSW,,Switch2初公開,2,2025-01-16,47560.0
2025-03-02,2025-02-28,PS5,,MHWs,1,2025-02-28,108978.0
2025-03-02,2025-02-28,PS5,,PS5新生活セール,2,2025-02-28,109978.0
2025-03-30,2025-03-27,NSW,,Nintendo Direct,3,2025-03-27,30489.0
2025-04-06,2025-04-02,NSW,,Switch2 Direct,1,2025-04-02,35041.0
2025-06-01,2025-05-28,PS5,,Days of Play2025,2,2025-05-28,15266.0
2025-06-08,2025-06-05,NSW,,Switch2発売,1,2025-06-05,16611.0
2025-07-20,2025-07-17,NS2,,DKバナンザ,2,2025-07-17,152165.0


In [11]:
df = hs.load_hard_sales()
df = hs.pivot_sales_by_delta(df)
event_df = he.load_hard_event()
event_df = he.delta_event(event_df, hi.load_hard_info())
filtered_events = he.add_event_positions_delta(event_df, df, priority=3)
filtered_events.sort_values(by='x_pos', ascending=True, inplace=True)
display(filtered_events)

,event_date,hw,hw2,event_name,priority,delta_week,x_pos,y_pos
report_date,,,,,,,,
2025-07-20,2025-07-17,NS2,,DKバナンザ,2,6,6,152165.0
2017-07-23,2017-07-21,NSW,,スプラ2,2,20,20,102581.0
2017-10-29,2017-10-27,NSW,,マリオデ,1,34,34,133094.0
2021-08-01,2021-08-01,PS5,,PS5 CFI1100,3,37,37,22641.0
2022-10-02,2022-10-01,PS5,,PS5 CFI1200,3,98,98,18076.0
2019-09-22,2019-09-20,NSW,,SwitchLite,1,133,133,239740.0
2016-09-11,2016-09-07,PS4,,PS4Slim,1,133,133,364.0
2023-06-25,2023-06-22,PS5,,FF16,1,136,136,85385.0
2023-08-06,2023-08-01,PS5,,PS5値上げ,1,142,142,50358.0
